In [3]:
import sys
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')
sys.path.append("../src")

In [4]:
import json

from utils import data_utils
from utils import conceptset_utils

In [ ]:
from pathlib import Path
root = Path.cwd().parent.parent

import os
os.chdir(root)

In [ ]:
!ls


In [4]:
"""
CLASS_SIM_CUTOFF: Concenpts with cos similarity higher than this to any class will be removed
OTHER_SIM_CUTOFF: Concenpts with cos similarity higher than this to another concept will be removed
MAX_LEN: max number of characters in a concept

PRINT_PROB: what percentage of filtered concepts will be printed
"""

CLASS_SIM_CUTOFF = 0.85
OTHER_SIM_CUTOFF = 0.9
MAX_LEN = 30
PRINT_PROB = 1

dataset = "cifar10"
device = "cuda"

save_name = "data/concept_sets/{}_filtered_new.txt".format(dataset)

In [5]:
#EDIT these to use the initial concept sets you want

with open(root / "data/concept_sets/gpt3_init/gpt3_{}_important.json".format(dataset), "r") as f:
    important_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_superclass.json".format(dataset), "r") as f:
    superclass_dict = json.load(f)
with open(root / "data/concept_sets/gpt3_init/gpt3_{}_around.json".format(dataset), "r") as f:
    around_dict = json.load(f)
    
with open(root / data_utils.LABEL_FILES[dataset], "r") as f:
    classes = f.read().split("\n")

In [6]:
concepts = set()

for values in important_dict.values():
    concepts.update(set(values))

for values in superclass_dict.values():
    concepts.update(set(values))
    
for values in around_dict.values():
    concepts.update(set(values))

print(len(concepts))

170


In [7]:
concepts = conceptset_utils.remove_too_long(concepts, MAX_LEN, PRINT_PROB)

170 170


In [8]:
concepts = conceptset_utils.filter_too_similar_to_cls(concepts, classes, CLASS_SIM_CUTOFF, device, PRINT_PROB)

170
Class:bird - Deleting bird
Class:cat - Deleting a cat
Class:deer - Deleting a deer
167
Class:airplane - Concept:a plane, sim:0.876 - Deleting a plane

Class:airplane - Concept:aircraft, sim:0.887 - Deleting aircraft

Class:automobile - Concept:a car, sim:0.881 - Deleting a car

Class:automobile - Concept:car, sim:0.921 - Deleting car

Class:automobile - Concept:vehicle, sim:0.904 - Deleting vehicle

Class:dog - Concept:animal, sim:0.855 - Deleting animal

Class:truck - Concept:a truck driver, sim:0.863 - Deleting a truck driver

160


In [11]:
concepts = conceptset_utils.filter_too_similar(concepts, OTHER_SIM_CUTOFF, device, PRINT_PROB)

ConnectionError: HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/models/sentence-transformers/all-mpnet-base-v2 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7faae5f7d910>: Failed to establish a new connection: [Errno 101] Network is unreachable'))

In [ ]:
with open(save_name, "w") as f:
    f.write(concepts[0])
    for concept in concepts[1:]:
        f.write("\n" + concept)